In [53]:
from __future__ import division

In [54]:
import numpy as np
import numpy.random as random
import matplotlib.pyplot as plt
import cv2

In [55]:
base_sign_path = './signs/'
base_target_path = './signs/target'
def add_path_prefix(c):
    return "0" + str(c)

In [56]:
import os

In [57]:
from skimage.transform import rotate
from skimage.transform import warp
from skimage.transform import ProjectiveTransform
from skimage import img_as_ubyte
import skimage.transform as ski_transform
import skimage.color as ski_color
from skimage import io
random.seed(12345)

def img_rotate(X, intensity):
    delta = 30. * intensity # scale using augmentation intensity
    X = rotate(X, random.uniform(-delta, delta), mode = 'edge')
    return X  

def apply_projection_transform(X, intensity):
    image_size = X.shape[1]
    d = image_size * 0.3 * intensity
    tl_top = random.uniform(-d, d)     # Top left corner, top margin
    tl_left = random.uniform(-d, d)    # Top left corner, left margin
    bl_bottom = random.uniform(-d, d)  # Bottom left corner, bottom margin
    bl_left = random.uniform(-d, d)    # Bottom left corner, left margin
    tr_top = random.uniform(-d, d)     # Top right corner, top margin
    tr_right = random.uniform(-d, d)   # Top right corner, right margin
    br_bottom = random.uniform(-d, d)  # Bottom right corner, bottom margin
    br_right = random.uniform(-d, d)   # Bottom right corner, right margin

    transform = ProjectiveTransform()
    transform.estimate(np.array((
            (tl_left, tl_top),
            (bl_left, image_size - bl_bottom),
            (image_size - br_right, image_size - br_bottom),
            (image_size - tr_right, tr_top)
        )), np.array((
            (0, 0),
            (0, image_size),
            (image_size, image_size),
            (image_size, 0)
        )))
    X = warp(X, transform, output_shape=(image_size, image_size), order = 1, mode = 'edge')

    return X
from skimage import data, io
from matplotlib import pyplot as plt


# img = io.imread('./signs/00/sign-giving-order-stop-give-way.jpg')
# img = ski_transform.resize(img, (32,32))
# img = ski_color.rgb2grey(img)
# img = apply_projection_transform(img, 0.8)

# io.imsave('./signs/test.png', img)
# io.imshow(img)
# plt.show()

In [58]:
# img=io.imread('./signs/target/00/65.png', as_grey=True)
# print (img)
# img = img_as_ubyte(img)
# img = img_as_ubyte(img)
# img = img_as_ubyte(img)
# img = img_as_ubyte(img)
# img = (img/255).astype(np.float32)
# print (img)

In [59]:
num_class = 10
img_width = 32
img_height = 32

In [60]:
def image_flip():
    horizontal_flippable = np.array([5, 9])
    verticle_flippable = np.array([5, 9])
    both_flippable = np.array([5, 9])
    cross_flippable = np.array([
        [1, 2],
        [3, 4],
        [2, 1],
        [4, 3]
    ])
    for cur_class in range(num_class):
        class_base_path = base_target_path + "/" + add_path_prefix(cur_class) + "/"
        os.mkdir(class_base_path)
    for cur_class in range(num_class):
        list_files=os.listdir(base_sign_path + "/" + add_path_prefix(cur_class))
        class_base_path = base_target_path + "/" + add_path_prefix(cur_class) + "/"
#         os.mkdir(class_base_path)
        img_iter = 0
        for index, cur_file in enumerate(list_files):
            img = io.imread(base_sign_path + "/" + add_path_prefix(cur_class) + "/" + cur_file)            
            img = ski_transform.resize(img, (32, 32))
            oimg = ski_color.rgb2gray(img)
            oimg = img_as_ubyte(oimg)
            img_iter+=1
            io.imsave(class_base_path + str(img_iter) + ".png", oimg)
#             k = cv2.waitKey(100000) & 0xFF
#             if k == 27:
#                 break
            if cur_class in horizontal_flippable:
#                 print "hor ",num_class
                img_iter+=1
                io.imsave(class_base_path + str(img_iter) + ".png", cv2.flip(oimg, 1))                    
            if cur_class in verticle_flippable:
#                 print "ver ",num_class
                img_iter+=1
                io.imsave(class_base_path + str(img_iter) + ".png", cv2.flip(oimg, 0))                    
            if cur_class in both_flippable:
#                 print "both ", num_class
                img_iter+=1
                io.imsave(class_base_path + str(img_iter) + ".png", cv2.flip(cv2.flip(oimg, 1), 0))                    
            if cur_class in cross_flippable[:,0]:
                img_iter+=1
                cross_class=cross_flippable[cross_flippable[:,0]==cur_class][0][1]
#                 print "cross ", cur_class, cross_class
                cross_path = base_target_path + "/" +add_path_prefix(cross_class) + "/"
                io.imsave(cross_path + str(img_iter) + ".png", cv2.flip(oimg, 1))
                
                
def rotate_data():
    for cur_class in range(num_class):
        class_base_path=base_target_path + "/" + add_path_prefix(cur_class) + "/"
        list_files=os.listdir(class_base_path)
        img_iter = len(list_files) - 1
        for cur_file in list_files:
            img = io.imread(base_target_path + "/" + add_path_prefix(cur_class) + "/" + cur_file,as_grey=True)
            for i in range(20):
                new_img = img_rotate(img, 0.4)
                img_iter+=1
                new_img = img_as_ubyte(new_img)
                io.imsave(class_base_path + str(img_iter) + ".png", new_img)
                
def project_data():
    for cur_class in range(num_class):
        class_base_path=base_target_path + "/" + add_path_prefix(cur_class) + "/"
        list_files=os.listdir(class_base_path)
        img_iter = len(list_files) - 1
        for cur_file in list_files:
            img = io.imread(base_target_path + "/" + add_path_prefix(cur_class) + "/" + cur_file, as_grey=True)
            for i in range(1):              
                new_img = apply_projection_transform(img, 0.6) 
                img_iter+=1
                new_img = img_as_ubyte(new_img)
                io.imsave(class_base_path + str(img_iter) + ".png", new_img)
                
!rm -rf ./signs/target/
!mkdir ./signs/target
image_flip()
rotate_data()
project_data()
# cv2.destroyAllWindows()

/opt/python-virtualenv/ml-python3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/opt/python-virtualenv/ml-python3/lib/python3.5/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
